In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

In [2]:
def f(pp, ind):
  return pp[:,ind,:]

In [3]:
lr = 0.0001
bs=32
dev = "cuda"
ep=50

In [7]:
df1 = pd.read_csv('/content/drive/MyDrive/archive/AI_Human.csv')

In [8]:
df1.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [9]:
def shuf(u1, u2):
  rm = np.arange(u1.shape[0])
  np.random.shuffle(rm)
  return (u1[rm], u2[rm])

In [10]:
X, y = df1.text.to_numpy(), df1.generated.to_numpy()

In [11]:
X_test, y_test = np.concat([X[y==0][3000:3050], X[y==1][3000:3050]], axis=0), np.concat([y[y==0][3000:3050], y[y==1][3000:3050]], axis=0)
X, y = np.concat([X[y==0][:400], X[y==1][:400]], axis=0), np.concat([y[y==0][:400], y[y==1][:400]], axis=0)
X, y = shuf(X, y)
del df1

In [12]:
from re import sub
x = []
for i in X:
  x.append(' '.join(sub(r'[^A-Za-z0-9]', " ", i).split()).lower())

In [13]:
from copy import copy as cp
n = len(x)
print(n)
s = set()
for i in range(n):
    t = x[i].split()
    yy = []
    for j in t:
      s.add(j)
      yy.append(j)
    x[i] = cp(yy)
print(len(s))
print(len(x))

800
9640
800


In [14]:
rr = len(s)+1
print(rr)

9641


In [15]:
t = list(map(len, x))
tt = max(t)
print(tt)

1337


In [16]:
d = {'': 0}
ss = list(s)
for i in range(rr-1):
  d[ss[i]] = i+1
del ss

In [17]:
for i in range(len(x)):
  for j in range(len(x[i])):
    x[i][j] = d[x[i][j]]

In [18]:
for i in range(len(x)):
  while len(x[i])<tt:
    x[i].append(0)

In [19]:
x_train = np.array(x)
x_train.shape

(800, 1337)

In [20]:
y_train =y
y_train.shape

(800,)

In [21]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False, categories = [list(range(rr)) for _ in range(tt)])
enc.fit(x_train)
x_train = enc.transform(x_train).reshape(x_train.shape[0], x_train.shape[1], rr)

In [22]:
del X; del y;

In [23]:
import gc
gc.collect()

78

test

In [24]:
from re import sub
x = []
for i in X_test:
  x.append(' '.join(sub(r'[^A-Za-z0-9]', " ", i).split()).lower())

In [25]:
from copy import copy as cp
n = len(x)
print(n)
s = set()
for i in range(n):
    t = x[i].split()
    yy = []
    for j in t:
      s.add(j)
      yy.append(j)
    x[i] = cp(yy)
print(len(s))
print(len(x))

100
3544
100


In [26]:
po=0
for i in range(len(x)):
  for j in range(len(x[i])):
    if x[i][j] not in d:
      x[i][j]=0
      po+=1
    else:
      x[i][j] = d[x[i][j]]
po

1119

In [27]:
for i in range(len(x)):
  while len(x[i])<tt:
    x[i].append(0)

In [28]:
x_test = np.array(x)
x_test.shape

(100, 1337)

In [29]:
x_test = enc.transform(x_test).reshape(x_test.shape[0], x_test.shape[1], rr)

In [30]:
x_test.shape

(100, 1337, 9641)

In [31]:
class RN(nn.Module):
  def __init__(self, one_hot_size, emb_size, n_steps):
    super(RN, self).__init__()
    self.one_hot_size, self.emb_size, self.n_steps = one_hot_size, emb_size, n_steps
    self.h = nn.ModuleList(); self.h.append(nn.Linear(one_hot_size, emb_size))
    for i in range(1, n_steps+1):
      self.h.append(nn.Linear(one_hot_size+emb_size, emb_size))
    self.h.append(nn.Linear(emb_size, emb_size))
    for i in range(2, n_steps+1):
      self.h.append(nn.Linear(emb_size+emb_size, emb_size))
    #self.fc1 = nn.Linear(emb_size*n_steps, 2000)
    #self.fc2 = nn.Linear(2000, 500)
    self.sm = nn.Linear(emb_size*n_steps, 1)
  def forward(self, x):
    n = self.n_steps
    z = [F.tanh(self.h[0](f(x, 0)))]
    for i in range(1, n+1):
      z.append(F.tanh(self.h[i](torch.cat((f(x, i-1), z[i-1]), dim=1))))
      if i>1:
        z[-1] = z[-1]+z[-3]
    z.append(F.tanh(self.h[n+1](z[n]))+z[n-1])
    for i in range(2, n+1):
      z.append(F.tanh(self.h[n+i](torch.cat((z[n-i+1], z[n+i-1]), dim=1))) + z[n-i] + z[n+i-2])
    o_emb = z[n+1]
    for i in range(2, n+1):
      o_emb = torch.cat((o_emb, z[n+i]), dim=1)
    #z.append(F.relu(self.fc1(o_emb)))
    #z.append(F.relu(self.fc2(z[-1])))
    z.append(self.sm(o_emb))
    return z[-1].reshape(z[-1].shape[0])

In [32]:
gc.collect()
net = RN(rr, 32,tt)
net.to(dev)
gc.collect()

55

In [33]:
net

RN(
  (h): ModuleList(
    (0): Linear(in_features=9641, out_features=32, bias=True)
    (1-1337): 1337 x Linear(in_features=9673, out_features=32, bias=True)
    (1338): Linear(in_features=32, out_features=32, bias=True)
    (1339-2674): 1336 x Linear(in_features=64, out_features=32, bias=True)
  )
  (sm): Linear(in_features=42784, out_features=1, bias=True)
)

In [34]:
crit = nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(net.parameters(), lr=0.00001)

In [37]:
def vl():
  net.eval()
  with torch.no_grad():
    cur, tot = 0, 0
    for i in range(len(x_test)//bs+(len(x_test)%bs!=0)):
      a = torch.from_numpy(x_test[i*bs:i*bs+bs]).float()
      b = torch.from_numpy(y_test[i*bs:i*bs+bs]).float()
      a = a.to(dev)
      b = b.to(dev)
      out = net(a)
      #print(out)
      tot+=b.shape[0]
      for i in range(b.shape[0]):
        if nn.Sigmoid()(out[i])>=0.5 and b[i]==1 or nn.Sigmoid()(out[i])<0.5 and b[i]==0:
          cur+=1
    print('accuracy:', cur/tot)
def vl2():
  net.eval()
  with torch.no_grad():
    cur, tot = 0, 0
    for i in range(len(x_train)//bs+(len(x_train)%bs!=0)):
      a = torch.from_numpy(x_train[i*bs:i*bs+bs]).float()
      b = torch.from_numpy(y_train[i*bs:i*bs+bs]).float()
      a = a.to(dev)
      b = b.to(dev)
      out = net(a)
      #print(out)
      tot+=b.shape[0]
      for i in range(b.shape[0]):
        if nn.Sigmoid()(out[i])>=0.5 and b[i]==1 or nn.Sigmoid()(out[i])<0.5 and b[i]==0:
          cur+=1
  print('accuracy:', cur/tot)

In [38]:
vl()
vl2()

accuracy: 0.55
accuracy: 0.51


In [40]:
q = []
tot = len(x_train)
pp = len(x_train)//bs+(len(x_train)%bs!=0)
for i in range(ep):
    net.train()
    for j in range(pp):
      a = torch.from_numpy(x_train[j*bs:j*bs+bs]).float()
      b = torch.from_numpy(y_train[j*bs:j*bs+bs]).float()
      a = a.to(dev)
      b = b.to(dev)
      out = net(a)
      loss = crit(out, b)
      #print(out, b)
      opt.zero_grad()
      loss.backward()
      opt.step()
      nn.utils.clip_grad_norm(net.parameters(), 1)
      gc.collect()
      if(j+1)%3==0:
        q.append(loss.item())
        print('Epoch {}/{} Step {}/{} LossL{:.4f}'.format(i+1, ep, j+1, pp, loss.item()))
        vl()
        vl2()
        net.train()
    #scheduler.step()
    vl()
plt.scatter(range(len(q)), q, s=3, c='black')

<ipython-input-40-2f2fa3f9b7cf>:17: FutureWarning: `torch.nn.utils.clip_grad_norm` is now deprecated in favor of `torch.nn.utils.clip_grad_norm_`.
  nn.utils.clip_grad_norm(net.parameters(), 1)


Epoch 1/50 Step 3/25 LossL10.2390
accuracy: 0.69
accuracy: 0.53125
Epoch 1/50 Step 6/25 LossL11.6338
accuracy: 0.61
accuracy: 0.53875
Epoch 1/50 Step 9/25 LossL10.3884
accuracy: 0.68
accuracy: 0.5725
Epoch 1/50 Step 12/25 LossL7.1159
accuracy: 0.7
accuracy: 0.59
Epoch 1/50 Step 15/25 LossL7.9065
accuracy: 0.71
accuracy: 0.6225
Epoch 1/50 Step 18/25 LossL7.1481
accuracy: 0.67
accuracy: 0.635
Epoch 1/50 Step 21/25 LossL6.0801
accuracy: 0.74
accuracy: 0.6525
Epoch 1/50 Step 24/25 LossL7.1141
accuracy: 0.73
accuracy: 0.6825
accuracy: 0.75
Epoch 2/50 Step 3/25 LossL1.6822
accuracy: 0.73
accuracy: 0.7175
Epoch 2/50 Step 6/25 LossL2.9739
accuracy: 0.72
accuracy: 0.74125
Epoch 2/50 Step 9/25 LossL2.5691
accuracy: 0.64
accuracy: 0.72875
Epoch 2/50 Step 12/25 LossL1.6657
accuracy: 0.67
accuracy: 0.70875
Epoch 2/50 Step 15/25 LossL1.8836
accuracy: 0.65
accuracy: 0.775
Epoch 2/50 Step 18/25 LossL3.3253
accuracy: 0.66
accuracy: 0.7525
Epoch 2/50 Step 21/25 LossL1.7532
accuracy: 0.74
accuracy: 0.73


KeyboardInterrupt: 

In [ ]:
a = torch.randn(7)
m = nn.Threshold(0.1, 20)
print(a)
print(torch.clamp(a, max=0.7))